In [1]:
import numpy as np
import tensorflow as tf

In [2]:
data = open('dinos.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
def word_to_one_vector(word, char_to_ix):
    one_hot_vectors = np.zeros((len(word), len(char_to_ix)))
    one_hot_vectors[ np.arange(len(word)), [char_to_ix[ch] for ch in word] ] = 1
    return one_hot_vectors

In [5]:
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]

In [6]:
n_neurons = 150

In [7]:
X = tf.placeholder(tf.float32, [None, None, vocab_size])
y = tf.placeholder(tf.int32, [None, None])
seq_len = tf.placeholder(tf.int32)

In [8]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units =  n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype = tf.float32, sequence_length = seq_len)

In [9]:
stacked_outputs = tf.reshape(outputs, [-1, n_neurons])
stacked_outputs_dense = tf.layers.dense(stacked_outputs, vocab_size)
outputs_2 = tf.reshape(stacked_outputs_dense, [-1, seq_len, vocab_size])

In [10]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = outputs_2)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
training_op = optimizer.minimize(loss)

In [11]:
init = tf.global_variables_initializer()

In [16]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(30000):
        for iteration, example  in enumerate(examples):
            x_ = np.vstack((np.zeros(len(char_to_ix)),word_to_one_vector(example, char_to_ix)))
            x_ = x_.reshape(1, x_.shape[0], x_.shape[1])
            y_ = np.append(np.asarray([(char_to_ix[x]) for x in example]),0)
            y_ = y_.reshape(1, y_.shape[0])
            sess.run(training_op, feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]})

            if iteration % 1000 == 0:
                print(loss.eval(feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]}))
                out = sess.run(outputs_2, feed_dict = {X: np.zeros((10,len(char_to_ix))).reshape(1,10,len(char_to_ix)), seq_len: 10})
                indexs = tf.argmax(tf.nn.softmax(out), 1).eval()[0]
                word = ''.join([ ix_to_char[x] for x in indexs])
                print(word)
            


2.252021
hi
c
ic
i
ce

ii
ididieibd

2.7082021
aa
h



aa





b
df
a
aaah
3.2773316
ca
c


a
a

b
ccb
cc
b





2.8719535

a
dba

aa





b
cc


bbaa
2.957856
aa
bba
caa

bbbcb
bbba


a

2.7385397

a
bbabaaa

c


b
bbca
aaaa
2.8918536
bh
bba
baaabbbbcb
bbba


a



KeyboardInterrupt: 